# Running a 5-Point Calibration Scan With OSO

This notebook adapts the Five Point Calibration Scan notebook to work with PSI. Running this notebook requires that a PSI namespace be spun up with `DISH_LMC_ENABLED` set to true.

This notebook can be Run with BITE or SPFRx, with BITE requiring the optional BITE notebook to be run. 

###  For using BITE

 BITE Functionality has been spun out to a separate notebook, which should be referred to when needed. These sections are labeled as BITE STEP.

## 1 Setup 

### 1.1 Environment Setup

Start by importing all the libraries needed to run this notebook, including local ones.

In [ ]:
import re
import sys

sys.path.append("../../src")

import json
import os
from time import sleep

from IPython.display import clear_output
from ska_oso_pdm import SBDefinition
from ska_oso_scripting import oda_helper
from ska_oso_scripting.functions import pdm_transforms
from ska_oso_scripting.functions.sb import create_sbi, load_sbd
from ska_oso_scripting.objects import SubArray, Telescope
from tango import DevFailed, DeviceProxy

### 1.2 Set Variables

First, grab the namespace launched from the pipeline:

In [ ]:
!kubectl get ns | grep ska-mid-psi 

And load it into the variables for the notebook:

In [ ]:
namespace = "ci-ska-mid-psi-1453068847-alexschell"  # set to desired NS
simulation_mode = 0  # set to 1 to run in sim mode
target_boards_list = [3]  # assign boards
scheduling_block_definition = "mid_5_point_sb.json"  # SB def file in data/scheduling_block to use
server = "ska-sdp-kafka." + namespace + ".svc.cluster.local:9092"
subarray_id = 1

With the namespace set, use these links to monitor the system as the notebook is worked through

In [ ]:
print("Monitor signal received by the vis pod:")
print(f"https://142.73.34.170/{namespace}/signal/display/")
print("Dashboard for monitoring device status:")
print(f"https://142.73.34.170/{namespace}/taranta/dashboard?id=666cb28b5e5d4f0012197e5f&mode=run")
print("Dashboard for monitoring TMC status:")
print(f"https://142.73.34.170/{namespace}/taranta/dashboard?id=669ea8e32bc4790019e64bba&mode=run")
print("PgAdmin interface to monitor the databases and ensure SB/EB are properly set in them")
print(f"https://142.73.34.170/{namespace}/pgadmin4/browser/")

Get the SKUID pod:

In [ ]:
!kubectl get svc -n $namespace | grep skuid
!kubectl get svc -n $namespace | grep kafka

And set the id value based on the name of this pod:

In [ ]:
skuid_id = "ska-ser-skuid-test-1406480615-svc"
kafka_id = "ska-sdp-kafka"

As a work around step for SKUID pod access, get the IP from the pod, get the IP address of the SKUID pod then set that as our access url:

In [ ]:
skuid_name = !kubectl -n $namespace get pods | grep ska-ser-skuid-test
skuid_name = skuid_name[0].split()[0]
print(skuid_name)
extracted_ip = !kubectl -n $namespace describe pod $skuid_name | grep ips -A 1
ip = re.search("[0-9]{0,3}\.[0-9]{0,3}\.[0-9]{0,3}\.[0-9]{0,3}", str(extracted_ip))
print(ip.group(0))

kafka_name = !kubectl -n $namespace get pods | grep ska-sdp-kafka
kafka_name = kafka_name[0].split()[0]
print(kafka_name)
extracted_kafka_ip = !kubectl -n $namespace describe pod $kafka_name | grep ips -A 1
kafka_ip = re.search("[0-9]{0,3}\.[0-9]{0,3}\.[0-9]{0,3}\.[0-9]{0,3}", str(extracted_kafka_ip))
print(kafka_ip.group(0))

Copy the IP from this store it to be written later:

In [ ]:
SKUID_POD = ip.group(0) + ":9870"
KAFKA_POD = kafka_ip.group(0) + ":9092"

Next, load all the other vars the notebook will use. These should not need to be changed for this run.

In [ ]:
# Tango host environment variable
TANGO_HOST = "databaseds-tango-base." + namespace + ".svc.cluster.local:10000"
# Don't use for now...
# SKUID_POD = skuid_id + "." + namespace + ".svc.cluster.local:5004"

# Parent directory to use to grab config files.
DATA_DIR = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), "data")
# Config file directories
COMMON_CONFIG = os.path.join(DATA_DIR, "mid_telescope/cbf")
HW_CONFIG = os.path.join(DATA_DIR, "mid_telescope/cbf/hw_config")
SB_DIR = os.path.join(DATA_DIR, "scheduling_block")

# For mapping the talon boards to receptor
RECEPTOR_MAP = ["SKA001", "SKA036", "SKA063", "SKA100"]

Next, set the environment arg for TANGO HOST, SKUID_POD and KAFKA, along with the oda API service endpoint:

In [ ]:
os.environ["TANGO_HOST"] = TANGO_HOST
os.environ["SKUID_URL"] = SKUID_POD
os.environ["ODA_URL"] = f"http://142.73.34.170/{namespace}/oda/api/v6/"
os.environ["telescope"] = "mid"
print("SKUID service set to: ", os.environ["SKUID_URL"])
print("TANGO_HOST set to:", os.environ["TANGO_HOST"])
print("ODA endpoint set to:{}".format(os.environ["ODA_URL"]))
print("This can be checked via {}ui".format(os.environ["ODA_URL"]))

With all the file paths defined the JSON files can be loaded in and checked.

In [ ]:
print("Getting files...")

DISH_CONFIG_FILE = f"{COMMON_CONFIG}/sys_params/load_dish_config.json"
SB_FILE = os.path.join(SB_DIR, scheduling_block_definition)

SCAN_COMBOS = [[0.0, 5.0], [0.0, -5.0], [5.0, 0.0], [-5.0, 0.0]]

files = [
    DISH_CONFIG_FILE,
    SB_FILE,
]

for file in files:
    if os.path.isfile(file):
        print(f"{file} exists: ✔️")
    else:
        print(f"{file} does not exist ❌")

Next, set up the hw config to match the boards in use:

In [ ]:
if any(i > 4 for i in target_boards_list):
    print("Using swap for higher number talons")
    config = "hw_config_swap_psi.yaml"
    print("Modifying target to use lower nums to match swap file")
    target_boards_list = list(map(lambda x: x - 4, target_boards_list))

else:
    print("Using standard HW config")
    config = "hw_config_psi.yaml"

HW_CONFIG_FILE = os.path.join(HW_CONFIG, config)
if os.path.isfile(HW_CONFIG_FILE):
    print("HW config: ✔️")
else:
    print("hw config: ❌")

receptor_ids = list(map(lambda x: RECEPTOR_MAP[x - 1], target_boards_list))

If the HW config file exists, it is loaded into the pod:

In [ ]:
!kubectl cp $HW_CONFIG_FILE $namespace/ds-cbfcontroller-controller-0:/app/mnt/hw_config/hw_config.yaml 

<style>
    .alert {
        background-color: #1a1d21;
        border-style: dotted;
        border-color: #f0493e;
        color: #d1d2d3;
    }
</style>
<div class="alert">
    <h3>(BITE STEP) Set Variables in the BITE Notebook</h3>
    If running the BITE notebook, at this point ensure the variables in step 1.2 are set, and match the ones set here.
</div>
</body>

### 1.3 Create Device Proxies

With the pod spun up, create device proxies to the devices used and check the connection to them.

In [ ]:
# CSP Devices
csp_controller = DeviceProxy("mid-csp/control/0")
print(f"CSP Controller: {csp_controller.Status()}")

# TMC Devices
tmc_central_node = DeviceProxy("ska_mid/tm_central/central_node")
print(f"Central Node: {tmc_central_node.Status()}")
tmc_subarray = DeviceProxy("ska_mid/tm_subarray_node/1")
print(f"TMC subarray Node: {tmc_subarray.Status()}")

# For checking leaf node
leaf_node_master = DeviceProxy("ska_mid/tm_leaf_node/csp_master")

# Deployer for setup and BITE for data mocking
deployer = DeviceProxy("mid_csp_cbf/ec/deployer")

# CBF Device
cbf_subarray = DeviceProxy("mid_csp_cbf/sub_elt/subarray_01")
print(f"CBF subarray: {cbf_subarray.Status()}")

### 1.4 Downloading Requirements via the Deployer

First, set the board to deploy to and turn on the deployer device.

In [ ]:
deployer.targetTalons = target_boards_list
print("Deployer will target board:", deployer.targetTalons)
deployer.generate_config_jsons()

Once started and configured, the required devices can then be downloaded.

In [ ]:
deployer.set_timeout_millis(400000)
try:
    deployer.download_artifacts()
except DevFailed as e:
    print(e)
    print(
        "Timed out, this is likely due to the download taking some time. Check the logs with the code space below after some time to see if it passes."
    )
deployer.set_timeout_millis(3000)

Now configure the device database with the downloaded devices:

In [ ]:
deployer.configure_db()

### 1.5 Set up Execution Block

The first step is to generate an empty Execution Block, using the helper scripts to work with the API endpoint:

In [61]:
exec_block_id = oda_helper.create_eb(telescope="ska_mid")
print(exec_block_id)

eb-t0001-20240913-00025


In [62]:
exec_block_id_2 = oda_helper.create_eb(telescope="ska_mid")
print(exec_block_id_2)

eb-t0001-20240913-00026


## 2 Preparing Telescope

Start by setting the admin modes to allow for control of the telescope:

In [ ]:
csp_controller.write_attribute("adminMode", 0)
tmc_central_node.write_attribute("adminMode", 0)
# Set CBF Simulation mode to false and CBF timeout to 99s
sleep(2)
csp_controller.cbfSimulationMode = simulation_mode
csp_controller.commandTimeout = 99

Ensure that these are set correctly, admin mode should be 0 (online) and cbfSimulationMode should be FALSE unless running in sim mode is desired.

In [ ]:
print(csp_controller.adminMode)
print(tmc_central_node.adminMode)
print(csp_controller.cbfSimulationMode)

Start by loading in the VCC config (this step may need to be re-run):

In [ ]:
with open(DISH_CONFIG_FILE, encoding="utf-8") as f:
    dish_config_json = json.load(f)

dish_config_json["tm_data_sources"][
    0
] = "car://gitlab.com/ska-telescope/ska-telmodel-data?0.1.0-rc-mid-itf#tmdata"
dish_config_json["tm_data_filepath"] = "instrument/ska1_mid_itf/ska-mid-cbf-system-parameters.json"

print(f"dish_config_json file contents: \n{dish_config_json}")
tmc_central_node.LoadDishCfg(json.dumps(dish_config_json))
# Wait for dishvcc to be loaded...
wait_seconds = 0
while not tmc_central_node.isDishVccConfigSet:
    clear_output(wait=True)
    print(f"Waiting for DishVCC to be set, {wait_seconds} seconds elapsed...")
    sleep(2)
    wait_seconds += 2

clear_output(wait=True)
print("DishVCC has been set!")
print(f"TMC CSP Master's Dish Vcc Config attribute value: \n{leaf_node_master.dishVccConfig}")
print(
    f"\nTMC CSP Master's Source Dish Vcc Config attribute value: \n{leaf_node_master.sourceDishVccConfig}"
)

<style>
    .alert {
        background-color: #1a1d21;
        border-style: dotted;
        border-color: #f0493e;
        color: #d1d2d3;
    }
</style>
<div class="alert">
    <h3>(BITE STEP) Load In BITE Config Data</h3>
    Now, if required, the BITE configuration data can be loaded in using the BITE notebook.
</div>
</body>

Set up the objects for controlling the telescope and subarray via OSO:

In [ ]:
sub = SubArray(subarray_id)
tel = Telescope()

Now, while monitoring the Taranta dashboard, send the ON command to the telescope and monitor that the LURs Controller and subarrays come on (occasionally the ON command may not finish when everything has started):

In [ ]:
tel.on()

<style>
    .alert {
        background-color: #1a1d21;
        border-style: dotted;
        border-color: #f0493e;
        color: #d1d2d3;
    }
</style>
<div class="alert">
    <h3>(BITE STEP) Generate BITE Data</h3>
    With the telescope on, section 3 of the BITE notebook can be run, to generate the BITE data that will be used during the LSTV replay.
</div>
</body>

### 2.1 Define Scheduling Block

Temporary work around for ingress issues:

In [ ]:
os.environ["SKUID_URL"] = SKUID_POD
print("SKUID service set to: ", os.environ["SKUID_URL"])

Now, use the SKUID service to generate processing and execution block IDs from the scheduling block definition.

In [63]:
sb = load_sbd(SB_FILE)
# Configure sb items for current board:
sb.sdp_configuration.resources.receptors = receptor_ids
sb.dish_allocations.receptor_ids = receptor_ids
# for now run with 1 fsp
sb.csp_configurations[0].cbf.fsps[0].fsp_id = target_boards_list[0]
sb.csp_configurations[0].cbf.fsps[0].frequency_slice_id = target_boards_list[0]

sbi = create_sbi(sb)

sbi.sdp_configuration.execution_block.eb_id = exec_block_id

1|2024-09-13T21:59:21.763Z|INFO|MainThread|create_sbi|sb.py#28||New SBI ID mapping: sbi-mvp01-20200325-00001 -> sbi-t0001-20240913-00027
1|2024-09-13T21:59:21.767Z|INFO|MainThread|create_sbi|sb.py#41||New PB ID mapping: pb-test-20211111-00000 -> pb-t0001-20240913-00028
1|2024-09-13T21:59:21.768Z|INFO|MainThread|create_sbi|sb.py#41||New PB ID mapping: pb-test-20211111-00001 -> pb-t0001-20240913-00029


In [64]:
SB_FILE_2 = os.path.join(SB_DIR, "mid_5_point_sb_2.json")

In [65]:
sb_2 = load_sbd(SB_FILE_2)
# Configure sb items for current board:
sb_2.sdp_configuration.resources.receptors = receptor_ids
sb_2.dish_allocations.receptor_ids = receptor_ids
# for now run with 1 fsp
sb_2.csp_configurations[0].cbf.fsps[0].fsp_id = target_boards_list[0]
sb_2.csp_configurations[0].cbf.fsps[0].frequency_slice_id = target_boards_list[0]

sbi_2 = create_sbi(sb_2)

sbi_2.sdp_configuration.execution_block.eb_id = exec_block_id_2

1|2024-09-13T21:59:28.855Z|INFO|MainThread|create_sbi|sb.py#28||New SBI ID mapping: sbi-mvp01-20200325-00002 -> sbi-t0001-20240913-00031
1|2024-09-13T21:59:28.861Z|INFO|MainThread|create_sbi|sb.py#41||New PB ID mapping: pb-test-20211111-00002 -> pb-t0001-20240913-00033


In [66]:
print(sb)
print(sbi)
print(sb_2)
print(sbi_2)

interface='https://schema.skao.int/ska-oso-pdm-sbd/0.1' sbd_id='sbi-mvp01-20200325-00001' name=None description=None telescope=TelescopeType.SKA_MID metadata=Metadata(version=1, created_by='Liz Bartlett', created_on=datetime.datetime(2022, 3, 28, 15, 43, 53, 971548, tzinfo=TzInfo(UTC)), last_modified_by='Liz Bartlett', last_modified_on=datetime.datetime(2023, 11, 15, 10, 45, 3, 313536, tzinfo=TzInfo(UTC))) prj_ref=None activities={'observe': GitScript(function_args={'init': PythonArguments(args=[], kwargs={'subarray_id': 1}), 'main': PythonArguments(args=[], kwargs={})}, kind=ScriptKind.GIT, repo='https://gitlab.com/ska-telescope/oso/ska-oso-scripting', path='git://scripts/test_example_mid_script.py', branch='master', commit=None)} targets=[Target(target_id='Polaris Australis', pointing_pattern=PointingPattern(active=PointingKind.FIVE_POINT, parameters=[FivePointParameters(kind=PointingKind.FIVE_POINT, offset_arcsec=5.0), RasterParameters(kind=PointingKind.RASTER, row_length_arcsec=1.2

Using this, create the JSON for the scheduling block, subbing in the Kafka location, and validating it.

In [67]:
sb_json = SBDefinition.model_dump_json(sbi)
sb_json = sb_json.replace("<KAFKA_HOST>", KAFKA_POD)
sbi = SBDefinition.model_validate_json(sb_json)

In [68]:
sb_json_2 = SBDefinition.model_dump_json(sbi_2)
sb_json_2 = sb_json_2.replace("<KAFKA_HOST>", KAFKA_POD)
sbi_2 = SBDefinition.model_validate_json(sb_json_2)

## 3 Running the Observation

### 3.1 Assigning Resources

Now, the resources can be assigned to the telescope, either via getting the resources from the scheduling block:

In [69]:
assign_requests = pdm_transforms.create_cdm_assign_resources_request_from_scheduling_block(
    sub.id, sbi
)

1|2024-09-13T22:00:02.249Z|INFO|MainThread|create_mid_assign_resources_request|wrapper.py#83||Setting dish : frozenset({'SKA001'}) 
1|2024-09-13T22:00:02.250Z|INFO|MainThread|convert_channels|sdp.py#294||Setting ChannelConfiguration id:vis_channels , spectral_windows:[SpectralWindow(spectral_window_id='fsp_1_channels', count=14880, start=0, freq_min=269000000.0, freq_max=469000000.0, stride=1, link_map=[(0, 0), (200, 1), (744, 2), (944, 3)])] 
1|2024-09-13T22:00:02.250Z|INFO|MainThread|convert_spectral_window|sdp.py#273||Setting channel attribute -> count:14880 , start:0 , stride:1, freq_min:269000000.0, freq_max:469000000.0 , link_map:[(0, 0), (200, 1), (744, 2), (944, 3)] 
1|2024-09-13T22:00:02.252Z|INFO|MainThread|convert_target_to_fieldconfiguration|sdp.py#348||Setting target attribute -> target_id:Polaris Australis , reference_coordinate:<EquatorialCoordinates: (21h08m47.92s -88d57m22.9s ICRS)> 
1|2024-09-13T22:00:02.254Z|INFO|MainThread|convert_target_to_fieldconfiguration|sdp.py

In [70]:
assign_requests_2 = pdm_transforms.create_cdm_assign_resources_request_from_scheduling_block(
    sub.id, sbi_2
)

1|2024-09-13T22:00:05.419Z|INFO|MainThread|create_mid_assign_resources_request|wrapper.py#83||Setting dish : frozenset({'SKA001'}) 
1|2024-09-13T22:00:05.421Z|INFO|MainThread|convert_channels|sdp.py#294||Setting ChannelConfiguration id:vis_channels , spectral_windows:[SpectralWindow(spectral_window_id='fsp_1_channels', count=14880, start=0, freq_min=269000000.0, freq_max=469000000.0, stride=1, link_map=[(0, 0), (200, 1), (744, 2), (944, 3)])] 
1|2024-09-13T22:00:05.421Z|INFO|MainThread|convert_spectral_window|sdp.py#273||Setting channel attribute -> count:14880 , start:0 , stride:1, freq_min:269000000.0, freq_max:469000000.0 , link_map:[(0, 0), (200, 1), (744, 2), (944, 3)] 
1|2024-09-13T22:00:05.422Z|INFO|MainThread|convert_target_to_fieldconfiguration|sdp.py#348||Setting target attribute -> target_id:Polaris Australis , reference_coordinate:<EquatorialCoordinates: (21h08m47.92s -88d57m22.9s ICRS)> 
1|2024-09-13T22:00:05.423Z|INFO|MainThread|convert_target_to_fieldconfiguration|sdp.py

In [71]:
print(sbi)
print(assign_requests)
print(sbi_2)
print(assign_requests_2)

interface='https://schema.skao.int/ska-oso-pdm-sbd/0.1' sbd_id='sbi-t0001-20240913-00027' name=None description=None telescope=TelescopeType.SKA_MID metadata=Metadata(version=1, created_by='Liz Bartlett', created_on=datetime.datetime(2022, 3, 28, 15, 43, 53, 971548, tzinfo=TzInfo(UTC)), last_modified_by='Liz Bartlett', last_modified_on=datetime.datetime(2023, 11, 15, 10, 45, 3, 313536, tzinfo=TzInfo(UTC))) prj_ref=None activities={'observe': GitScript(function_args={'init': PythonArguments(args=[], kwargs={'subarray_id': 1}), 'main': PythonArguments(args=[], kwargs={})}, kind=ScriptKind.GIT, repo='https://gitlab.com/ska-telescope/oso/ska-oso-scripting', path='git://scripts/test_example_mid_script.py', branch='master', commit=None)} targets=[Target(target_id='Polaris Australis', pointing_pattern=PointingPattern(active=PointingKind.FIVE_POINT, parameters=[FivePointParameters(kind=PointingKind.FIVE_POINT, offset_arcsec=5.0), RasterParameters(kind=PointingKind.RASTER, row_length_arcsec=1.2

In [72]:
print(assign_requests)
stripped_assign_requests = assign_requests
stripped_assign_requests.interface = ""
stripped_assign_requests.dish.receptor_ids = receptor_ids
print(stripped_assign_requests)

subarray_id=1 dish=DishAllocation(receptor_ids=frozenset({'SKA001'})) csp_config=None sdp_config=SDPConfiguration(interface='https://schema.skao.int/ska-sdp-assignres/0.4', transaction_id=None, execution_block=ExecutionBlockConfiguration(eb_id='eb-t0001-20240913-00025', max_length=21600.0, context={}, beams=[BeamConfiguration(beam_id='vis0', function='visibilities', search_beam_id=None, timing_beam_id=None, vlbi_beam_id=None)], channels=[ChannelConfiguration(channels_id='vis_channels', spectral_windows=[Channel(count=14880, start=0, stride=1, freq_min=269000000.0, freq_max=469000000.0, link_map=[[0, 0], [200, 1], [744, 2], [944, 3]], spectral_window_id='fsp_1_channels')])], polarisations=[PolarisationConfiguration(polarisations_id='all', corr_type=['XX', 'XY', 'YX', 'YY'])], fields=[FieldConfiguration(field_id='Polaris Australis', pointing_fqdn=None, phase_dir=PhaseDir(ra=[317.19966666666664], dec=[-88.95636111111111], reference_time='2024-09-13T22:00:02.252987+00:00', reference_frame=

In [73]:
print(assign_requests_2)
stripped_assign_requests_2 = assign_requests_2
stripped_assign_requests_2.interface = ""
stripped_assign_requests_2.dish.receptor_ids = receptor_ids
print(stripped_assign_requests_2)

subarray_id=1 dish=DishAllocation(receptor_ids=frozenset({'SKA001'})) csp_config=None sdp_config=SDPConfiguration(interface='https://schema.skao.int/ska-sdp-assignres/0.4', transaction_id=None, execution_block=ExecutionBlockConfiguration(eb_id='eb-t0001-20240913-00026', max_length=21600.0, context={}, beams=[BeamConfiguration(beam_id='vis0', function='visibilities', search_beam_id=None, timing_beam_id=None, vlbi_beam_id=None)], channels=[ChannelConfiguration(channels_id='vis_channels', spectral_windows=[Channel(count=14880, start=0, stride=1, freq_min=269000000.0, freq_max=469000000.0, link_map=[[0, 0], [200, 1], [744, 2], [944, 3]], spectral_window_id='fsp_1_channels')])], polarisations=[PolarisationConfiguration(polarisations_id='all', corr_type=['XX', 'XY', 'YX', 'YY'])], fields=[FieldConfiguration(field_id='Polaris Australis', pointing_fqdn=None, phase_dir=PhaseDir(ra=[317.19966666666664], dec=[-88.95636111111111], reference_time='2024-09-13T22:00:05.423055+00:00', reference_frame=

Now assign the resources to the subarray.

In [74]:
sub.assign_from_cdm(stripped_assign_requests, timeout=1000)

1|2024-09-13T22:01:06.495Z|INFO|MainThread|_call_and_wait_for_transition|common.py#139||Using pub/sub to track obsState of ska_mid/tm_subarray_node/1
1|2024-09-13T22:01:06.499Z|INFO|MainThread|_get_id_from_params_or_generate_new_id|transactions.py#172||Generated transaction ID txn-t0001-20240913-000000035
1|2024-09-13T22:01:06.499Z|INFO|MainThread|log_entry|transactions.py#132||Transaction[txn-t0001-20240913-000000035]: Enter[AssignResources] with parameters [{}] marker[12853]
1|2024-09-13T22:01:06.500Z|INFO|MainThread|_call_and_wait_for_transition|common.py#170||Added Transaction_id to command: <Command('ska_mid/tm_central/central_node', 'AssignResources', '{"subarray_id": 1, "dish": {"receptor_ids": ["SKA001"]}, "sdp": {"interface": "https://schema.skao.int/ska-sdp-assignres/0.4", "execution_block": {"eb_id": "eb-t0001-20240913-00026", "max_length": 21600.0, "context": {}, "beams": [{"beam_id": "vis0", "function": "visibilities"}], "channels": [{"channels_id": "vis_channels", "spectr

Check that the visibility pod spins up correctly, with the signal displays.

In [ ]:
!kubectl -n $namespace-sdp get pods | grep vis-receive

proc-pb-t0001-20240913-00029-vis-receive-00-0                   6/6     Running     2 (12m ago)   12m


In [89]:
sub.assign_from_cdm(stripped_assign_requests_2, timeout=1000)

1|2024-09-13T22:13:54.010Z|INFO|MainThread|_call_and_wait_for_transition|common.py#139||Using pub/sub to track obsState of ska_mid/tm_subarray_node/1
1|2024-09-13T22:13:54.015Z|INFO|MainThread|_get_id_from_params_or_generate_new_id|transactions.py#172||Generated transaction ID txn-t0001-20240913-000000037
1|2024-09-13T22:13:54.015Z|INFO|MainThread|log_entry|transactions.py#132||Transaction[txn-t0001-20240913-000000037]: Enter[AssignResources] with parameters [{}] marker[81990]
1|2024-09-13T22:13:54.015Z|INFO|MainThread|_call_and_wait_for_transition|common.py#170||Added Transaction_id to command: <Command('ska_mid/tm_central/central_node', 'AssignResources', '{"subarray_id": 1, "dish": {"receptor_ids": ["SKA001"]}, "sdp": {"interface": "https://schema.skao.int/ska-sdp-assignres/0.4", "execution_block": {"eb_id": "eb-t0001-20240913-00026", "max_length": 21600.0, "context": {}, "beams": [{"beam_id": "vis0", "function": "visibilities"}], "channels": [{"channels_id": "vis_channels", "spectr

KeyboardInterrupt: 

In [87]:
!kubectl -n $namespace-sdp get pods | grep vis-receive

proc-pb-t0001-20240913-00029-vis-receive-00-0                   6/6     Running     2 (6m55s ago)   7m15s


<style>
    .alert {
        background-color: #1a1d21;
        border-style: dotted;
        border-color: #f0493e;
        color: #d1d2d3;
    }
</style>
<div class="alert">
    <h3>(BITE STEP) Starting LSTV Replay</h3>
    If using BITE for data stream generation, the additional BITE notebook can be run at this point.
</div>
</body>

### 3.3 Run the Scan(s)

To start running the scans (or just one) first create the configuration request from the scheduling block:

In [ ]:
scan_def_to_config_req_mapping = pdm_transforms.create_cdm_configure_request_from_scheduling_block(
    sbi
)

In [ ]:
print(scan_def_to_config_req_mapping)
config_req = scan_def_to_config_req_mapping["science scan"]
print(config_req[0])

#### Running a Single Scan

For a single scan, load in part of the scan configuration just for that run, and send the configure commands to the subarray. Once this is complete, run the scan command itself. Using the scheduling block sets how long the scan will run, so it will finish by itself.

In [ ]:
print(scan_def_to_config_req_mapping)
config_req = scan_def_to_config_req_mapping["science scan"]
print(config_req)
sub.configure_from_cdm(config_req[0], timeout=500)
sleep(5)
sub.scan()

#### Running Multiple Scans

To run multiple scans, iterate over the scan sequence, and use the ids in it to define a request and then send it to the subarray.

In [ ]:
print("...")
print(scan_def_to_config_req_mapping)
print("...")
for scan_def_id in sb.scan_sequence:
    config_req = scan_def_to_config_req_mapping[scan_def_id]
    print(config_req)
    print("--------------")
    for request in config_req:
        sub.configure_from_cdm(request, timeout=500)
        sleep(2)
        sub.scan()

## 4 Cleanup

<style>
    .alert {
        background-color: #1a1d21;
        border-style: dotted;
        border-color: #f0493e;
        color: #d1d2d3;
    }
</style>
<div class="alert">
    <h3>(BITE STEP) Stopping LSTV Replay</h3>
    Now with the initial (or only) scan done, if using BITE, run step 5 in the BITE notebook to stop the LSTV Replay.
</div>
</body>

Once the scans are done, clear out the configuration from the subarray by sending the END command, followed by releasing the resources it used:

In [ ]:
sub.end()
sleep(10)
sub.release()

The telescope can now be turned off.

In [ ]:
tel.off()